# M026 - 2 Folds trained seperately
## Merge The Results together

In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [9]:
sub_f1 = pd.read_csv('../submissions/M026_0629_1637_submission_lgb_2folds_1.3690CV_500000iter_0.1lr.csv')
sub_f2 = pd.read_csv('../submissions/M026_0629_1638_submission_lgb_2folds_1.3679CV_500000iter_0.1lr.csv')

In [3]:
oof_f1 = pd.read_csv('../oof/M026_0629_1637_oof_lgb_2folds_1.3690CV_500000iter_0.1lr.csv')
oof_f2 = pd.read_csv('../oof/M026_0629_1638_oof_lgb_2folds_1.3679CV_500000iter_0.1lr.csv')

In [4]:
fi_f1 = pd.read_csv('../fi/M026_0629_1637_fi_lgb_2folds_1.3690CV_500000iter_0.1lr.csv')
fi_f2 = pd.read_csv('../fi/M026_0629_1638_fi_lgb_2folds_1.3679CV_500000iter_0.1lr.csv')

# Merge

In [5]:
ss = pd.read_csv('../input/sample_submission.csv')

In [6]:
ss['f1'] = sub_f1['scalar_coupling_constant']
ss['f2'] = sub_f2['scalar_coupling_constant']

In [10]:
ss['scalar_coupling_constant'] = ss[['f1','f2']].sum(axis=1) # SUM BECAUSE THEY WERE DIVIDED WHEN CALCULATED RAW

In [11]:
ss = ss.drop(['f1','f2'], axis=1)

In [31]:
oof = oof_f1[['id','type','scalar_coupling_constant']].copy()

In [32]:
oof['oof_f1'] = oof_f1['oof_preds']
oof['oof_f2'] = oof_f2['oof_preds']

In [12]:
oof['oof'] = oof[['oof_f1','oof_f2']].sum(axis=1)

In [13]:
for i, d in oof.groupby('type'):
    print('Type {} - MAE {:0.4f}'.format(i, mean_absolute_error(d['scalar_coupling_constant'], d['oof'])))

Type 1JHC - MAE 0.7054
Type 1JHN - MAE 0.4130
Type 2JHC - MAE 0.3348
Type 2JHH - MAE 0.1883
Type 2JHN - MAE 0.1618
Type 3JHC - MAE 0.4452
Type 3JHH - MAE 0.2295
Type 3JHN - MAE 0.1905


In [14]:
group_mean_log_mae(oof['scalar_coupling_constant'], oof['oof'], oof['type'])

-1.2197825237349829

In [15]:
fi = pd.concat([fi_f1, fi_f2])

# Save Results

In [39]:
ss.to_csv('../submissions/M026_COMBINED_submission_lgb_2folds_-1.2198CV_500000iter_0.1lr.csv', index=False)
oof.to_csv('../oof/M026_COMBINED_oof_lgb_2folds_-1.2198CV_500000iter_0.1lr.csv', index=False)
fi.to_csv('../fi/M026_COMBINED_fi_lgb_2folds_-1.2198CV_500000iter_0.1lr.csv', index=False)